In [1]:
from utils.t5 import *

In [2]:
input_model_name = "d-t5-source_text_shorter_LOF_base_0.11_data_explanation_prep_4.pickle"
input_data_name = "LOF_base_0.11_data_explanation_prep_4.pickle"  

input_model_dir = "./Data/Models/"
input_data_dir = "./Data/Selection/"
output_dir = "./Data/Generation/"
source_column = "source_text_shorter"          #source_text_shorter       #"statement_explanation_prep"                          #"statement_explanation_prep"  "explanation_prep"   # "source_text_shorter" "source_text"
generated_column = "automatic_text"                           # "automatic_text"
model_type = "t5"
batch_size = 30 # 50
no_workers = 6

input_data_path = "{}{}".format(input_data_dir, input_data_name)
new_data_path = "{}generated_{}.pickle".format(output_dir,input_model_name)
input_model_path = "{}{}".format(input_model_dir,input_model_name)

In [36]:
torch.cuda.get_device_name(0)

'A100-SXM4-40GB'

In [38]:
data = pd.read_pickle(input_data_path)

In [39]:
train, dev_test = train_test_split(data, test_size = 0.2, random_state = 42)
dev, test = train_test_split(dev_test, test_size = 0.5, random_state = 42)

In [41]:
import gc
#del data, model
gc.collect()
import torch
torch.cuda.empty_cache()

In [42]:
model = SimpleT5()

In [43]:
input_model_path

'./Data/Models/d-t5-source_text_shorter_LOF_base_0.11_data_explanation_prep_4.pickle'

In [44]:
model.load_model(model_type, input_model_path, use_gpu = True)

In [48]:
input_data = test[source_column].to_list()
print(len(input_data))
automatic_lst = model.predict(input_data, batch_size = batch_size)

1290


  0%|          | 0/43 [00:00<?, ?it/s]

In [49]:
new_data_path

'./Data/Generation/generated_d-t5-source_text_shorter_LOF_base_0.11_data_explanation_prep_4.pickle.pickle'

In [50]:
test[generated_column] = automatic_lst
print(len(test))
test.to_pickle(new_data_path)

1290


## Test of parallel processing 

### GPU

In [ ]:
from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass

data = test[source_column].to_list()[0:100]

cpus = 4 #multiprocessing.cpu_count()


def m_predict(n):
    model.predict(n)

pool  = Pool(cpus)
automatic_lst = pool.map(m_predict, tqdm(data))

In [ ]:
from torch.multiprocessing import set_start_method
set_start_method('spawn')

### CPU

In [ ]:
import multiprocessing

cpus = multiprocessing.cpu_count()
data = test[source_column].to_list()


def m_predict(n):
    model.predict(n)
    print("done")

pool = multiprocessing.Pool(processes=cpus)  # 
pool.map(m_predict, tqdm(data)) #automatic_lst = 

In [ ]:
#data = test[source_column].to_list()
#automatic_lst = [model.predict(i) for i in tqdm(data)]

In [ ]:
#test = test[["statement","label","explanation_prep","shortExplanation_prep"]]
test[generated_column] = automatic_lst
test.to_pickle(new_data_path)